In [1]:
from langchain.agents import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from typing import Dict, List, Optional, TypedDict


In [2]:
import sys

# sys.path.append("../")

from custom_features.models import MODELS
from custom_features.params_setting_fct import set_api_keys, set_langsmith

set_api_keys()
set_langsmith()

In [3]:

# 1. Définition des Outils
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Recherche sur Internet",
        func=search.run,
        description="utile pour lorsque vous avez besoin de répondre à des questions sur des sujets d'actualité. L'entrée doit être une requête de recherche.",
    ),
]


In [4]:

# 2. Définition des Agents (avec LLM et Prompt)
llm = ChatMistralAI(temperature=0)


# src/latest_ai_development/config/agents.yaml
researcher:
  role: >
    {topic} Senior Data Researcher
  goal: >
    Uncover cutting-edge developments in {topic}
  backstory: >
    You're a seasoned researcher with a knack for uncovering the latest
    developments in {topic}. Known for your ability to find the most relevant
    information and present it in a clear and concise manner.

reporting_analyst:
  role: >
    {topic} Reporting Analyst
  goal: >
    Create detailed reports based on {topic} data analysis and research findings
  backstory: >
    You're a meticulous analyst with a keen eye for detail. You're known for
    your ability to turn complex data into clear and concise reports, making
    it easy for others to understand and act on the information you provide.

# src/latest_ai_development/config/tasks.yaml
research_task:
  description: >
    Conduct a thorough research about {topic}
    Make sure you find any interesting and relevant information given
    the current year is 2025.
  expected_output: >
    A list with 10 bullet points of the most relevant information about {topic}
  agent: researcher

reporting_task:
  description: >
    Review the context you got and expand each topic into a full section for a report.
    Make sure the report is detailed and contains any and all relevant information.
  expected_output: >
    A fully fledge reports with the mains topics, each with a full section of information.
    Formatted as markdown without '```'
  agent: reporting_analyst
  output_file: report.md

In [23]:

prompt_researcher = ChatPromptTemplate.from_messages(
    [
        # (
        #     "system",
        #     "Role: {topic} Senior Data Researcher\n\n" \
        #     "Goal: Uncover cutting-edge developments in {topic}\n\n" \
        #     "Backstory: You're a seasoned researcher with a knack for uncovering the latest " \
        #     "developments in {topic}. Known for your ability to find the most relevant " \
        #     "information and present it in a clear and concise manner.\n\n" \
        #     "Description:\n" \
        #         "- Conduct a thorough research about {topic}\n" \
        #         "- Make sure you find any interesting and relevant information given\n" \
        #         "- The current year is 2025.\n\n" \
        #     "Expected output: A list with 10 bullet points of the most relevant information about {topic}"
        # ),
        (
            "system",
            "Role: Senior Financial Researcher/Analyst\n\n" \
            "Goal: Analyze financial markets and uncover strategic insights in {topic}\n\n" \
            "Backstory: You're an experienced financial analyst with a deep understanding of financial markets. " \
            "Known for your ability to identify key trends, evaluate financial data, and provide actionable insights.\n\n" \
            "Description:\n" \
                "- Conduct a comprehensive analysis of financial data related to {topic}\n" \
                "- Identify key trends, risks, and opportunities in the financial landscape\n" \
                "- Ensure the information is relevant and up-to-date as of 2025\n" \
                "- Present findings in a clear and concise manner\n\n" \
            "Expected output: A list with 10 bullet points of the most relevant financial insights about {topic}"
        ),
        ("user", "{topic}"),
    ]
)
prompt_reporting_analyst = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Role: {topic} Reporting Analyst\n\n" \
            "Goal: Create detailed reports based on {topic} data analysis and research findings\n\n" \
            "Backstory: You're a meticulous analyst with a keen eye for detail. You're known for " \
            "your ability to turn complex data into clear and concise reports, making " \
            "it easy for others to understand and act on the information you provide.\n\n" \
            "Description: " \
                "- Review the context you got and expand each topic into a full section for a report.\n" \
                "- Make sure the report is detailed and contains any and all relevant information.\n\n" \
            "Expected output: \n" \
                "- A fully fledge reports with the mains topics, each with a full section of information.\n" \
                "- Formatted as markdown without '```'"
        ),
        ("user", "Informations provided by {topic} Senior Data Researcher: {informations}\n\nUser question: {topic}"),
    ]
)


In [24]:

# Agent Chercheur MODIFIÉ pour retourner un dictionnaire mettant à jour l'état avec 'informations'
def agent_chercheur_node(state):
    question = state['topic']
    resultat_recherche = search.run(question) # Utilisation de l'outil de recherche
    return {"informations": resultat_recherche} # Retourne un dictionnaire pour mettre à jour l'état

# Agent Répondeur MODIFIÉ (légèrement) pour accéder à 'informations' depuis l'état
def agent_repondeur_node(state):
    informations = state['informations']
    question = state['topic']
    prompt_avec_informations = prompt_researcher.format_messages(informations=informations, topic=question)
    reponse = llm.invoke(prompt_avec_informations)
    return {"reponse": reponse.content} # Retourne la réponse, pourrait aussi mettre à jour l'état si nécessaire


In [25]:

# 3. Définition du Graph LangGraph
class AgentState(TypedDict):
    topic: str
    informations: Optional[str]
    reponse: Optional[str] # 'reponse' sera mis à jour par le répondeur

builder = StateGraph(AgentState)

# Ajout des nodes (agents)
builder.add_node("chercheur", agent_chercheur_node)
builder.add_node("repondeur", agent_repondeur_node)

# Définition des edges (transitions)
builder.set_entry_point("chercheur")
builder.add_edge("chercheur", "repondeur")
builder.add_edge("repondeur", END) # Fin du graph après le répondeur

# Compilation du Graph
graph = builder.compile()


In [30]:

# 4. Exécution du Workflow
inputs = {"topic": "TESLA"}
resultats = graph.invoke(inputs)


In [31]:

# 5. Affichage du résultat
reponse_finale = resultats['informations'] # La réponse du répondeur est stockée dans 'informations' dans cet exemple simplifié.
print(f"Question: {inputs['topic']}")
print(f"Réponse: {reponse_finale}")

Question: TESLA
Réponse: Tesla, Inc. is an American manufacturer of electric vehicles, solar panels, and automobile batteries. It was founded in 2003 by American entrepreneurs Martin Eberhard and Marc Tarpenning and was named after Serbian American inventor Nikola Tesla. Elon Musk, an early investor in the company, became CEO in 2008. Tesla has ambitious goals for its new Roadster: hit 60 mph in 1.9 seconds and achieve a range of 620 miles. Fittingly, prices will start around $200,000. Fittingly, prices will start around $200,000. 2024 Tesla Model 3 Long Range Rear-Wheel Drive: $44,130 The updated 2024 Tesla Model 3 is a solid upgrade over the outgoing model. It offers a quieter cabin, slight exterior tweaks, a more refined ... Overall, Tesla's profits took a hit in 2024, with net income dropping to $8.4 billion, marking a 23% drop from 2023 and an even steeper 40% decline from its record $14.1 billion profit in 2022 ... Tesla Motors continues to defy automotive norms, and the norm tha